In [60]:
import pandas as pd

In [61]:
enduti_usuarios_anual_2023=pd.read_csv(r'data\conjunto_de_datos_endutih_2023_csv\conjunto_de_datos\tr_endutih_usuarios_anual_2023.csv', dtype="object", encoding='latin1')

In [62]:
enduti_usuarios_anual_2023=enduti_usuarios_anual_2023[['SEXO','EDAD','ENT', 'ESTRATO','P7_21', 'FAC_PER']]

In [63]:
import pandas as pd

# Define the age bins and labels
age_bins = [ 17, 24, 30, 35, 45, 55, 65, float('inf')]
age_labels = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

# Arrays for each zone
noroeste = [2, 3, 8, 10, 25, 26]
noreste = [5, 19, 24, 28]
occidente_bajio = [1, 6, 11, 14, 16, 18, 22, 32]
cdmx = [9]
centro_sur_oriente = [13, 15, 21, 23, 29, 30]
sur = [4, 7, 12, 17, 20, 27, 31]

def assign_zone(ent):
    if ent in noroeste:
        return 'noroeste'
    elif ent in noreste:
        return 'noreste'
    elif ent in occidente_bajio:
        return 'occidente_y_bajio'
    elif ent in cdmx:
        return 'ciudad_de_mexico'
    elif ent in centro_sur_oriente:
        return 'centro_sur_y_oriente'
    elif ent in sur:
        return 'sur'
    else:
        return 'desconocido'  # In case an entity doesn't match any zone
enduti_usuarios_anual_2023['ENT'] = pd.to_numeric(enduti_usuarios_anual_2023['ENT'])
# Apply the function to create the 'ZONA' column
enduti_usuarios_anual_2023['ZONA'] = enduti_usuarios_anual_2023['ENT'].apply(assign_zone)

# Convert 'EDAD' to numeric
enduti_usuarios_anual_2023['EDAD'] = pd.to_numeric(enduti_usuarios_anual_2023['EDAD'])

# Create age groups
enduti_usuarios_anual_2023['GRUPO_EDAD'] = pd.cut(enduti_usuarios_anual_2023['EDAD'], bins=age_bins, labels=age_labels)

# Convert 'FAC_PER' to numeric
enduti_usuarios_anual_2023['FAC_PER'] = pd.to_numeric(enduti_usuarios_anual_2023['FAC_PER'])

# Group by relevant columns including the P7 column
enduti_agrupada = enduti_usuarios_anual_2023.groupby(['GRUPO_EDAD',  'ZONA', 'ESTRATO', 'P7_21'], observed=False).agg({'FAC_PER': 'sum'}).reset_index()

# Pivot the dataframe to have the P7 column values as separate columns
enduti_pivoteada = enduti_agrupada.pivot_table(
    index=['GRUPO_EDAD', 'ZONA', 'ESTRATO'], 
    columns='P7_21', 
    values='FAC_PER', 
    fill_value=0,
    observed=False
).reset_index()

# Rename columns for clarity
enduti_pivoteada.columns = ['GRUPO_EDAD',  'ZONA', 'ESTRATO', 'FAC_PER_P7_19_1', 'FAC_PER_P7_19_2']

In [64]:
#delete temp variables
del enduti_agrupada
del enduti_usuarios_anual_2023

In [65]:
enduti_pivoteada['POBTOT'] = enduti_pivoteada['FAC_PER_P7_19_1'] + enduti_pivoteada['FAC_PER_P7_19_2']
enduti_pivoteada

,GRUPO_EDAD,ZONA,ESTRATO,FAC_PER_P7_19_1,FAC_PER_P7_19_2,POBTOT
0,a,centro_sur_y_oriente,1,172901.0,618846.0,791747.0
1,a,centro_sur_y_oriente,2,1185173.0,1284562.0,2469735.0
2,a,centro_sur_y_oriente,3,438072.0,240886.0,678958.0
3,a,centro_sur_y_oriente,4,165335.0,70306.0,235641.0
4,a,ciudad_de_mexico,1,0.0,0.0,0.0
...,...,...,...,...,...,...
163,g,occidente_y_bajio,4,45001.0,112174.0,157175.0
164,g,sur,1,4025.0,114720.0,118745.0
165,g,sur,2,29830.0,209746.0,239576.0
166,g,sur,3,20603.0,95167.0,115770.0


In [66]:
enduti_pivoteada=enduti_pivoteada[enduti_pivoteada['POBTOT']>0]
enduti_pivoteada=enduti_pivoteada.sort_values(by=['FAC_PER_P7_19_2'], ascending=False)
enduti_pivoteada.head(40)

,GRUPO_EDAD,ZONA,ESTRATO,FAC_PER_P7_19_1,FAC_PER_P7_19_2,POBTOT
73,d,centro_sur_y_oriente,2,998859.0,1998620.0,2997479.0
97,e,centro_sur_y_oriente,2,506586.0,1922213.0,2428799.0
89,d,occidente_y_bajio,2,599602.0,1344586.0,1944188.0
1,a,centro_sur_y_oriente,2,1185173.0,1284562.0,2469735.0
113,e,occidente_y_bajio,2,348018.0,1164157.0,1512175.0
121,f,centro_sur_y_oriente,2,157570.0,1157783.0,1315353.0
25,b,centro_sur_y_oriente,2,940838.0,937193.0,1878031.0
17,a,occidente_y_bajio,2,952261.0,898187.0,1850448.0
72,d,centro_sur_y_oriente,1,137560.0,816529.0,954089.0
20,a,sur,1,258087.0,779277.0,1037364.0


In [67]:
enduti_pivoteada.to_csv('data/out/oportunidad_compradores_2023.csv', index=False)